[Specifying shader URL, with uniforms](https://docs.google.com/document/d/1QvCvey8HaUi6SP-p36ZVwh2NGqzB_cqU_KHrAj8yFtY/edit)

[Binder notebook update](http://mybinder.org/status/rsargent/notebooks)

(hal3 / anaconda)

In [1]:
import array, gzip, math, numpy, StringIO, struct, urllib, urllib2

def benchmark(label, data):
    zipped = StringIO.StringIO()
    zipper = gzip.GzipFile(fileobj = zipped, mode='wb')
    zipper.write(data)
    zipper.close()
    print "%.2fM / %.2fM: %s" % (len(zipped.getvalue()) / 1e6, len(data) / 1e6, label)

def encode_field(fieldspec, value):
    if 'min' in fieldspec and 'max' in fieldspec:
        value = (float(value) - fieldspec['min']) / (fieldspec['max'] - fieldspec['min'])
    if fieldspec['type'] == 'float':
        return struct.pack('<f', value)
    elif fieldspec['type'] == 'fix2':
        return struct.pack('<H', min(0xffff, max(0, int(round(value * 0xffff)))))
    elif fieldspec['type'] == 'fix3':
        return struct.pack('<I', min(0xffffff, max(0, int(round(value * 0xffffff)))))[0:3]
    else:
        raise Exception('encode_field giving up')

def decode_field(fieldspec, stream):
    if fieldspec['type'] == 'float':
        value = struct.unpack('<f', stream.read(4))[0]
    elif fieldspec['type'] == 'fix2':
        value = struct.unpack('<H', stream.read(2))[0] / float(0xffff)
    elif fieldspec['type'] == 'fix3':
        value = struct.unpack('<I', stream.read(3) + '\000')[0] / float(0xffffff)
    else:
        raise Exception('encode_field giving up')
    if 'min' in fieldspec and 'max' in fieldspec:
        value = value * (fieldspec['max'] - fieldspec['min']) + fieldspec['min']
    return value

def field_length(fieldspec):
    return len(encode_field(fieldspec, 0))

def spec_length(spec):
    ret = 0
    for fieldspec in spec['fields']:
        ret += field_length(fieldspec)
    return ret
        
def encode_row_major(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for r in range(0, len(data), nfields):
        for c in range(0, nfields):
            ret.append(encode_field(spec['fields'][c], data[r + c]))
    return ''.join(ret)

def decode_row_major(spec, stream, stream_length):
    nfields = len(spec['fields'])
    if stream_length % spec_length(spec) != 0:
        raise Exception('streamlength %d should be multiple of spec_length %d' % (stream_length, spec_length(spec)))
    ret = []
    for r in range(0, stream_length / spec_length(spec)):
        for c in range(0, nfields):
            ret.append(decode_field(spec['fields'][c], stream))
    return ret

def encode_col_major(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for c in range(0, nfields):
        for r in range(0, len(data), nfields):
            ret.append(encode_field(spec['fields'][c], data[r + c]))
    return ''.join(ret)

def decode_col_major(spec, stream, stream_length):
    nfields = len(spec['fields'])
    nrows = stream_length / spec_length(spec)
    if stream_length % spec_length(spec) != 0:
        raise Exception('streamlength %d should be multiple of spec_length %d' % (stream_length, spec_length(spec)))
    col_major = []
    for c in range(0, nfields):
        for r in range(0, nrows):
            col_major.append(decode_field(spec['fields'][c], stream))
    row_major = []
    for r in range(0, nrows):
        for c in range(0, nfields):
            row_major.append(col_major[c * nrows + r])
    return row_major

def encode_col_major_delta(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for c in range(0, nfields):
        field_spec = spec['fields'][c]
        if 'min' in field_spec and 'max' in field_spec:
            prev = (field_spec['min'] + field_spec['max']) / 2.0
        else:
            prev = 0.0
        for r in range(0, len(data), nfields):
            # ideal_delta is the value we want to encode
            ideal_delta = data[r + c] - prev
            encoded = encode_field(spec['fields'][c], ideal_delta)
            ret.append(encoded)
            # encoded_delta is the value we actually encoded, after rounding
            encoded_delta = decode_field(spec['fields'][c], StringIO.StringIO(encoded))
            prev = prev + encoded_delta
    return ''.join(ret)

def decode_col_major_delta(spec, stream, stream_length):
    nfields = len(spec['fields'])
    nrows = stream_length / spec_length(spec)
    if stream_length % spec_length(spec) != 0:
        raise Exception('streamlength %d should be multiple of spec_length %d' % (stream_length, spec_length(spec)))
    col_major = []
    for c in range(0, nfields):
        field_spec = spec['fields'][c]
        if 'min' in field_spec and 'max' in field_spec:
            prev = (field_spec['min'] + field_spec['max']) / 2.0
        else:
            prev = 0.0
        for r in range(0, nrows):
            current = decode_field(spec['fields'][c], stream) + prev
            col_major.append(current)
            prev = current
    row_major = []
    for r in range(0, nrows):
        for c in range(0, nfields):
            row_major.append(col_major[c * nrows + r])
    return row_major

def benchmark_encoding(label, spec, data):
    benchmark(label + ' row-major', encode_row_major(spec, data))
    benchmark(label + ' col-major', encode_col_major(spec, data))
    benchmark(label + ' col-major-delta', encode_col_major_delta(spec, data))

def analyze_columns(spec, data):
    ncols = len(spec['fields'])
    for c in range(0, ncols):
        vals = []
        for i in range(c, len(data), ncols):
            vals.append(data[i])
        print '%s: min %g, max %g, mean %g, stddev %g' % (spec['fields'][c]['name'],
            numpy.min(vals), numpy.max(vals), numpy.mean(vals), numpy.std(vals))
        
def compare_data(spec, a, b):
    if len(a) != len(b):
        raise Exception('a and b should be same length in compare_data')
    ncols = len(spec['fields'])
    for c in range(0, ncols):
        deltas = []
        for i in range(c, len(a), ncols):
            deltas.append(a[i] - b[i])
            if c == 0 and a[i] - b[i] > 0.1:
                print 'col 0 offset %d %g %g' % (i, a[i], b[i])
                return
        deltas = numpy.array(deltas)
        print '%s: error min %g, max %g, mean %g, stddev %g' % (spec['fields'][c]['name'],
            numpy.min(deltas), numpy.max(deltas), numpy.mean(deltas), numpy.std(deltas))

In [2]:
class GenericWebMercator(object):
    def __init__(self, west, north, east, south, width, height):
        self.west = west
        self.north = north
        self.east = east
        self.south = south
        self.width = width
        self.height = height
    
    @staticmethod
    def rawProjectLat(lat):
        return math.log((1.0 + math.sin(lat * math.pi / 180.0)) / math.cos(lat * math.pi / 180.0))

    @staticmethod
    def rawUnprojectLat(y):
        return (2.0 * math.atan(math.exp(y)) - math.pi / 2.0) * 180.0 / math.pi

    @staticmethod
    def interpolate(x, fromLow, fromHigh, toLow, toHigh):
        return (x - fromLow) / (fromHigh - fromLow) * (toHigh - toLow) + toLow

    def latlngToPoint(self, lat, lng):
        x = self.interpolate(lng, self.west, self.east, 0, self.width)
        y = self.interpolate(self.rawProjectLat(lat), self.rawProjectLat(self.north),
                             self.rawProjectLat(self.south), 0, self.height)
        return (x,y)

    def pointToLatlng(self, x, y):
        lng = self.interpolate(x, 0, self.width, self.west, self.east);
        lat = self.rawUnprojectLat(self.interpolate(y, 0, self.height, 
                                                    self.rawProjectLat(self.north), self.rawProjectLat(self.south)));
        return (lat, lng)

class StandardWebMercator(GenericWebMercator):
    def __init__(self):
        super(StandardWebMercator, self).__init__(-180.0, 85.05112877980659, 180.0, -85.05112877980659, 256.0, 256.0)


#def test(x, y):
#    proj = StandardWebMercator()
#    (lat, lng) = proj.pointToLatlng(x, y)
#    (newx, newy) = proj.latlngToPoint(lat, lng)
#    print "%g,%g -> %g,%g -> %g,%g" % (x, y, lat, lng, x, y)
#
#test(0,0)
#test(128,128)
#test(256,256)
#test(0,256)
#test(256,0)

In [3]:
xyz_time_bin = open('viirs-2014.timev').read()
benchmark("xyz_time float row-major", xyz_time_bin)
xyz_time = array.array('f', xyz_time_bin)

18.41M / 26.06M: xyz_time float row-major


In [4]:
lonlat_time_temp_bin = urllib2.urlopen('https://data.cmucreatelab.org/createlab-viirs/createlab-viirs-uncorrected-20140314-20150403.bin').read()
benchmark("mercatorxy_time_temp float row-major", lonlat_time_temp_bin)
lonlat_time_temp = array.array('f', lonlat_time_temp_bin)
# Convert Epoch-based time from milliseconds to seconds
for i in range(2, len(lonlat_time_temp), 4):
    lonlat_time_temp[i] /= 1000

# Convert xy mercator to lonlat
proj = StandardWebMercator()
for i in range(0, len(lonlat_time_temp), 4):
    (lonlat_time_temp[i + 1], lonlat_time_temp[i + 0]) = proj.pointToLatlng(lonlat_time_temp[i + 0], lonlat_time_temp[i + 1])

benchmark("lonlat_time_temp float row-major", lonlat_time_temp_bin)

spec_ffff = { 
    'stride': 16,
    'fields':[{'name':'lon', 'type':'float'},
              {'name':'lat', 'type':'float'},
              {'name':'time', 'type':'float'},
              {'name':'temp', 'type':'float'}]}


analyze_columns(spec_ffff, lonlat_time_temp)

15.16M / 26.06M: mercatorxy_time_temp float row-major
15.16M / 26.06M: lonlat_time_temp float row-major
lon: min -179.877, max 179.916, mean 31.4941, stddev 70.3391
lat: min -88.6965, max 88.328, mean 20.9391, stddev 26.8721
time: min 1.39477e+09, max 1.42812e+09, mean 1.41212e+09, stddev 9.44536e+06
temp: min 506, max 2500, mean 1319.43, stddev 358.935


In [5]:
lonlat_time = []
for i in range(0, len(lonlat_time_temp), 4):
    lonlat_time.append(lonlat_time_temp[i + 0])
    lonlat_time.append(lonlat_time_temp[i + 1])
    lonlat_time.append(lonlat_time_temp[i + 2])
lonlat_time_bin = array.array('f', lonlat_time).tostring()
benchmark("lonlat_time float row-major", lonlat_time_bin)

12.11M / 19.54M: lonlat_time float row-major


In [6]:
spec_fff = { 
    'stride': 12,
    'fields':[{'name':'lon', 'type':'float'},
              {'name':'lat', 'type':'float'},
              {'name':'time', 'type':'float'}]}

benchmark_encoding('lonlat_time fff', spec_fff, lonlat_time)

12.11M / 19.54M: lonlat_time fff row-major
11.85M / 19.54M: lonlat_time fff col-major
11.32M / 19.54M: lonlat_time fff col-major-delta


In [7]:
# lat and lon encoded as 24-bit fixed point, ~2m resolution
# (radius of earth * 2 * pi) / (2^24)
# time encoded as day number since posix epoch


# 23 bits for lat and lng, approx 5 meter resolution
spec_332 = { 
    'stride': 8,
    'fields':[{'name':'lon', 'type':'fix3', 'min':-360, 'max':360},
              {'name':'lat', 'type':'fix3', 'min':-180, 'max':180},
              {'name':'time', 'type':'fix2', 'min':-2831112000, 'max':2831112000}]}

benchmark_encoding('lonlat_time 332', spec_332, lonlat_time)

9.98M / 13.03M: lonlat_time 332 row-major
9.49M / 13.03M: lonlat_time 332 col-major
8.38M / 13.03M: lonlat_time 332 col-major-delta


In [8]:
# 22 bits for lat and lng, approx 10 meter resolution
spec_332 = { 
    'stride': 8,
    'fields':[{'name':'lon', 'type':'fix3', 'min':-720, 'max':720},
              {'name':'lat', 'type':'fix3', 'min':-360, 'max':360},
              {'name':'time', 'type':'fix2', 'min':-2831112000, 'max':2831112000}]}

benchmark_encoding('lonlat_time 332', spec_332, lonlat_time)

9.71M / 13.03M: lonlat_time 332 row-major
9.27M / 13.03M: lonlat_time 332 col-major
8.09M / 13.03M: lonlat_time 332 col-major-delta


In [9]:
# 21 bits for lat and lng, approx 20 meter resolution
spec_332 = { 
    'stride': 8,
    'fields':[{'name':'lon', 'type':'fix3', 'min':-1440, 'max':1440},
              {'name':'lat', 'type':'fix3', 'min':-720, 'max':720},
              {'name':'time', 'type':'fix2', 'min':-2831112000, 'max':2831112000}]}

benchmark_encoding('lonlat_time 332', spec_332, lonlat_time)

9.41M / 13.03M: lonlat_time 332 row-major
8.99M / 13.03M: lonlat_time 332 col-major
7.78M / 13.03M: lonlat_time 332 col-major-delta


In [10]:
# Test column major encoding

encoded = encode_col_major(spec_332, lonlat_time)
print len(encoded)
decoded = decode_col_major(spec_332, StringIO.StringIO(encoded), len(encoded))
print len(decoded)
compare_data(spec_332, lonlat_time, decoded)

13028448
4885668
lon: error min -8.58307e-05, max 8.58307e-05, mean 3.95415e-06, stddev 4.94447e-05
lat: error min -4.29153e-05, max 4.29152e-05, mean -5.21171e-08, stddev 2.47848e-05
time: error min -43200, max 43072, mean -9233.7, stddev 27570.8


In [11]:
# Test column major delta encoding

encoded = encode_col_major_delta(spec_332, lonlat_time)
print len(encoded)
decoded = decode_col_major_delta(spec_332, StringIO.StringIO(encoded), len(encoded))
print len(decoded)
compare_data(spec_332, lonlat_time, decoded)

13028448
4885668
lon: error min -8.58307e-05, max 8.58307e-05, mean 4.53593e-06, stddev 4.9399e-05
lat: error min -4.29153e-05, max 4.29153e-05, mean -7.02081e-09, stddev 2.47727e-05
time: error min -43200, max 43136, mean -830.825, stddev 25181.6


In [12]:
# Testing

foo = encode_row_major(spec_fff, lonlat_time[0:12])
print lonlat_time[0:12]
print decode_row_major(spec_fff, StringIO.StringIO(foo), len(foo))

foo = encode_row_major(spec_332, lonlat_time[0:12])
print len(foo)
print decode_row_major(spec_332, StringIO.StringIO(foo), len(foo))

field_spec = {'name':'foo', 'type':'fix2', 'min':0, 'max':5662224000}
f = encode_field(field_spec, 0)
print decode_field(field_spec, StringIO.StringIO(f))
f = encode_field(field_spec, 84600)
print " ".join("{:02x}".format(ord(c)) for c in f)
print decode_field(field_spec, StringIO.StringIO(f))

f = encode_field(field_spec, 1394773760.0)
print " ".join("{:02x}".format(ord(c)) for c in f)
print decode_field(field_spec, StringIO.StringIO(f))

field_spec = {'name':'lon', 'type':'fix3', 'min':-180, 'max':180}
f = encode_field(field_spec, 171.02980041503906)
print " ".join("{:02x}".format(ord(c)) for c in f)
print decode_field(field_spec, StringIO.StringIO(f))



[60.510658264160156, 67.65991973876953, 1394773760.0, 60.52228546142578, 67.6534652709961, 1394773760.0, 58.64055633544922, 68.29972076416016, 1394773760.0, 57.962215423583984, 68.61719512939453, 1394773760.0]
[60.510658264160156, 67.65991973876953, 1394773760.0, 60.52228546142578, 67.6534652709961, 1394773760.0, 58.64055633544922, 68.29972076416016, 1394773760.0, 57.962215423583984, 68.61719512939453, 1394773760.0]
32
[60.51072481338542, 67.65994952082337, 1394798400.0, 60.522226126326814, 67.65342638811035, 1394798400.0, 58.6404740000055, 68.29973151086165, 1394798400.0, 57.9622398592378, 68.6172192464602, 1394798400.0]
0.0
01 00
86400.0
0f 3f
1394755200.0
05 9f f9
171.029791297
